# Disassembled thermal circuits from folder describing the building

This notebook presents how to obtain _disassambled thermal circuits_ from a _folder describing the building_ implemented in the function `bldg2TCd()` of dm4bem module.

The _folder describing the building_ contains (see, for example, the folder `bldg`):

- description of the walls (wall type and wall data);
- description of thermal circuits;
- assembling matrix or list.

The _disassembled thermal network_ is a dictionary of thermal networks, each thermal network being described by the matrices A, G, C and the vectors b, f, y.

In [1]:
import pandas as pd
import pd_dm4bem

## Stating the problem
Consider the disassembled thermal circuits shown in Figure 1 that we want to assemble as shown in Figure 2.

![disassambled_TC](./bldg/ass_TCd.svg)
> Figure 1. Four disassembled thermal circuits: wall_out: ow0 (in red), TC0: c0 (in blue), TC1: c1 (in green), TC2: c2 (in magenta), TC3: c3 (in black).

![disassambled_TC](./bldg/ass_TC.svg)
> Figure 2. The assembling of the four circuits from Figure 1. The faded nodes are deleted.

The assembling is done by putting temperature nodes in common, e.g., the node 0 of the circuit c0 to be the same with the node 4 of the circuit ow0. This operation is called [merging of vertexes](https://en.m.wikipedia.org/wiki/Graph_operations#Unary_operations) in graph theory.

## Description of the folder containing the circuits
The disassembled circuits and the indications on how to assemble them are given in the folder `.\bldg` composed by the files:
- assembly:
    - `assembly_lists.csv`: lists that indicate the nodes to be merged;
    - `assembly_matrix.csv`: matrix that indicates the nodes to be merged;
- thermal circuits: `TC0.csv`, ... , `TC3.csv`;
- walls:
    - `wall_types.csv`: physical properties for each type of wall; 
    - `walls_out.csv`: surface properties for each specific wall. 

### Assembly matrix and lists

#### Assembly matrix

The assembly matrix is in the file `assembly_matrix.csv`. Each row of this matrix indicates two nodes that are put in common: the first node (designated by columns **TC0** and **node0**) is kept; the second node (designated by columns **TC1** and **node1**) is merged with the first node. For example, in Figure 2 the nodes put in commun are indicated by black dashed arrows. The tip of the arrow represents the node that is retained, while the tail of the arrow signifies the node that is merged. Within a node, there cannot coexist both a tip and a tail of the arrows indicating merging.

The five merging arrows in Figure 2 are:

0. Node 4 of circuit ow0 (in red) is in common with node 0 of circuit c0 (in blue); node 4 of circuit ow0 is kept (node 0 of circuit c0, in faded blue, is deleted).
1. 2nd last node of circuit c1 (in green) is in common with node 1 of circuit c0 (in blue); the node of circuit c1 is kept (node 1 of circuit c0, in faded blue, is deleted).
2. Node 0 of circuit c2 (in magenta) is in common with the last node of circuit ow0 (in red); the node of circuit c2 is kept (last node of circuit ow0, in faded red, is deleted).
3. Node 0 of circuit c2 (in magenta) is in common with the 1st node of circuit c3 (in black); the node of circuit c2 is kept (node 0 of circuit c3, in faded black, is deleted).
4. Node 0 of circuit c2 (in magenta) is in common with the last node of circuit c1 (in green); the node of circuit c2 is kept (last node of circuit c1, in faded green, is deleted).

These connexions are described in the file `assembly_matrix.csv`.

In [2]:
pd.read_csv('./bldg/assembly_matrix.csv')

,TC0,node0,TC1,node1
0,ow0,4,c0,0
1,c1,-2,c0,1
2,c2,0,ow0,-1
3,c2,0,c3,0
4,c2,0,c1,-1


__Note:__ Since a node cannot be, in the same time, deleted and kept, a node that appears in multiple rows cannot be specified in the 1st two columns on one row and in the last two columns on another row. For example, node 0 of circuit c2 (which is kept) can be only in the first columns.

#### Assembly lists
The assembly lists are in the file `assembly_lists.csv`. Each row contains two lists:
- __node0__: the node that is kept (to which all the merging arrows point);
- __nodes__: the node or nodes that are merged and deleted (that are the tail of the merging arrows).

For the circuits shown in Figures 1 and 2, there are three lists representing the three nodes that are kept (nodes to which the merging arrows point):
- On row 0:
    - node 0 of circuit c2, which is kept;
    - last node of circuit ow0, 1st node of circuit c3, last node of circuit c1, which are merged with node 0 of circuit c2.
- On row 1:
    - node 4 of circuit ow0, which is kept;
    - node 0 of circuit c0, which is merged.
- On row 2:
    - second last node of circuit c1, which is kept;
    - node 1 of circuit c0, which is merged.

In [3]:
pd.read_csv('./bldg/assembly_lists.csv')

,node0,nodes
0,"['c2', 0]","['ow0', -1], ['c3', 0], ['c1', -1]"
1,"['ow0', 4]","['c0', 0],"
2,"['c1', -2]","['c0', 1],"


### Thermal circuits

The names of the files which describe the thermal circuits start with TC, e.g., `TC0.csv`. A thermal circuit is a data structure composed of:
- **A**: incidence matrix  which indicates how the nodes are connected by oriented branches;
- **G**: conductance diagonal matrix ;
- **C**: capacity diagonal matrix ;
- **b**: temperature source vector ;
- **f**: heat flow source vector ;
- **y**: output (temperature) vector.

For example, the `TC1.csv` file describes the thermal circuit **c1** (in green) from Figures 1, 2 and 3.

![disassambled_TC](./bldg/TC1.svg)
> Figure 3. Thermal circuit `TC1: c1` described in the file `TC1.csv`.

In [4]:
df = pd.read_csv('./bldg/TC1.csv')
df

,A,θg,θwi,θai,G,b
0,qgo,1,NaN,NaN,165.789,To
1,qgi,-1,1.0,NaN,630.000,NaN
2,qai,NaN,-1.0,1.0,72.000,NaN
3,C,1089000.0,NaN,NaN,NaN,NaN
4,f,Φa,NaN,NaN,NaN,NaN
5,y,NaN,NaN,1.0,NaN,NaN


The incidence matrix __A__ describes how the temperature nodes _θ_ are connected by the oriented flow branches _q_:
- The rows `qgo`, `qgi`, `qai` (glass out, glass in, and air in, respectively) correspond to the flow branches.
- The columns `θg`, `θwi`, `θai` (glass, window surface in and air in, respectively) correspond to the temperature nodes.
- Value `1` means that the flow enters into the node, e.g., flow `qgo` enters node `θg`.
- Value `-1` means that the flow exists from the node, e.g., flow `qgi` quits node `θg`.
- Value `0` or no value (which is rendered as `NaN`by Pandas dataframe) means that the flow is not connected to the node, e.g., flow `qgo` not conected to node `θwi`.

In [5]:
df.loc[df['A'].isin(['qgo', 'qgi', 'qai']),
       ['A', 'θg', 'θwi', 'θai']]

,A,θg,θwi,θai
0,qgo,1,NaN,NaN
1,qgi,-1,1.0,NaN
2,qai,NaN,-1.0,1.0


The conductances, given in column __G__ for each branch _q_, are the values on the diagonal of the conductance matrix. The conductances __G__ are defined only for the rows corresponding to the flow branches. There are no values for __G__ in the last three rows (corresponding to C, f and y).

In [6]:
df.loc[df['A'].isin(['qgo', 'qgi', 'qai']), ['A', 'G']]

,A,G
0,qgo,165.789
1,qgi,630.000
2,qai,72.000


The capacities, given in row __C__ for each node _θ_, are the values on the diagonal of the capacity matrix.  The capacities __C__ are defined only for the columns corresponding to the temperature nodes. There are always no values for __C__ in the last two columns (corresponding to G and b). The capacities in some temperature nodes may be zero (or NaN).

In [7]:
df.loc[df['A'] == 'C',
       ['A', 'θg', 'θwi', 'θai']]

,A,θg,θwi,θai
3,C,1089000.0,NaN,NaN


Column __b__ contains the names of the temperature sources on each flow branch. The temperature sources __b__ are defined only for the rows corresponding to the flow branches.

In [8]:
df.loc[df['A'].isin(['qgo', 'qgi', 'qai']),
       ['A', 'b']]

,A,b
0,qgo,To
1,qgi,NaN
2,qai,NaN


Row __f__ contains the names of the flow sources in each temperature node. The flow sources __f__ are defined only for the columns corresponding to the temperature nodes.

In [9]:
df.loc[df['A'] == 'f',
       ['A', 'θg', 'θwi', 'θai']]

,A,θg,θwi,θai
4,f,Φa,NaN,NaN


Row __y__ indicates by "1" (ones) the temperature nodes that are the outputs of the model.

In [10]:
df.loc[df['A'] == 'y',
       ['A', 'θg', 'θwi', 'θai']]

,A,θg,θwi,θai
5,y,NaN,NaN,1.0


Note that in the data structure of a thermal circuit (the `.csv` file) there are always no values for the block defined by the rows __C__, __f__ and __y__ and the columns __G__ and __b__.

### Walls

The walls are characterized by two files: 
- wall composition and physical properties of material of types of walls (e.g., file `wall_types.csv`);
- surface characteristics of each wall (e.g., file `walls_out.csv`).  

#### Wall types

Let's consider two types of walls described in the file `wall_types.csv`.

In [11]:
pd.read_csv('./bldg/wall_types.csv')

,type,Material,Conductivity,Specific heat,Density,Width,Mesh
0,0,Concrete,1.400,880,2300,0.200,1
1,0,Insulation,0.027,1210,55,0.080,1
2,1,Glass,1.400,750,2500,0.004,1


The walls of __type__ 0 have two layers: Concrete and Insulation. Their physical characteristics (conductivity, specific heat, density) are given in SI units (W·m⁻¹·K⁻¹, J·kg⁻¹·K⁻¹, and kg·m⁻³, respectively). The width of the layers is in m. __Mesh__ represents the number of meshes in which the layer is discretized. Each mesh has two resistances and a capacity.

The wall __owO__ (in red) in Figures 1 and 2 is of type 0. It has two layers (concrete and insulation), each one discretized in one mesh.

#### Walls data

There are three types of plane walls (see _1.2 Walls data_ in [01wall2TC.ipynb](01wall2TC.ipynb)):
1. generic,
2. outdoor wall,
3. indoor wall.

In this folder, there is only one outdoor wall described by the file `walls_out.csv`. 

In [12]:
pd.read_csv('./bldg/walls_out.csv')

,ID,type,Area,β,γ,albedo,T0,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,0,45,90,0,0.2,To,Φo,Φi,25,8,0.25,0.3,0.85,0.7,[-1]


The outdoor wall with __ID__ w0 has the characteristics:
- __type__: 0, given in the file `wall_types.csv`.
- __Area__: 45 m², surface area of the plan wall;
- __β__: 90°, wall slope (0° horizontal, 90° vertical; > 90° downward facing);
- __γ__:  0°, azimuth (0° South, 180° North, >0 westward, <0 eastward);
- __T0__: 'To', name of the temperature source of outer surface;
- __Q0__: 'Qo', name of the flow rate source of outer surface;
- __Q1__: 'Qi', name of the flow rate source of inner surface;
- __h0__: 25 W·m⁻²·K⁻¹, convection coefficient of outer surface;
- __h1__: 8 W·m⁻²·K⁻¹, convection coefficient of inner surface;
- __α0__: 0.25, short-wave absorbtion coefficient of outer surface;
- __α1__: 0.3, short-wave absorbtion coefficient of inner surface;
- __ε0__: 0.85, long-wave hemispherical emissivity of outer surface;
- __ε1__: 0.7, long-wave hemispherical emissivity of inner surface;
- __y__:  [0, -1], output temperature nodes by using slicing, in this case the 1st node and the last node are outputs.


## Disassembled thermal circuits

The disassembled thermal circuits `TCd` are obtained from the folder `bldg` that describes the building.

In [13]:
print('\nDisassembled thermal circuits with auto-numbering')
TCd = pd_dm4bem.bldg2TCd(folder_path='bldg',
                        TC_auto_number=True)


Disassembled thermal circuits with auto-numbering


### Accessing the thermal circuits

The variable `TCd` is a dictionary; the keys of this dictionary are the names of the circuits. Each circuit from the set of disassembled thermal circuits `TCd` can be accessed.

In [14]:
TCd['c1']

{'A':        c1_θ0  c1_θ1  c1_θ2
 c1_q0    1.0    0.0    0.0
 c1_q1   -1.0    1.0    0.0
 c1_q2    0.0   -1.0    1.0,
 'G': c1_q0    165.789
 c1_q1    630.000
 c1_q2     72.000
 Name: G, dtype: float64,
 'C': c1_θ0    1089000.0
 c1_θ1          0.0
 c1_θ2          0.0
 Name: C, dtype: float64,
 'b': c1_q0    To
 c1_q1     0
 c1_q2     0
 Name: b, dtype: object,
 'f': c1_θ0     Φa
 c1_θ1    0.0
 c1_θ2    0.0
 Name: f, dtype: object,
 'y': c1_θ0    0
 c1_θ1    0
 c1_θ2    1
 Name: y, dtype: int64}

The elements (A, G, C, b, f, y) of each circuit can be accessed separately.

In [15]:
TCd['c1']['A']

,c1_θ0,c1_θ1,c1_θ2
c1_q0,1.0,0.0,0.0
c1_q1,-1.0,1.0,0.0
c1_q2,0.0,-1.0,1.0


In [16]:
TCd['c1']['G']

c1_q0    165.789
c1_q1    630.000
c1_q2     72.000
Name: G, dtype: float64

All circuits contained in the data structure `TCd` can be printed (see Figures 1 and 2).

In [17]:
for key in TCd.keys():
    print('Thermal circuit:', key)
    pd_dm4bem.print_TC(TCd[key])

Thermal circuit: ow0
A:
        ow0_θ0  ow0_θ1  ow0_θ2  ow0_θ3  ow0_θ4  ow0_θ5
ow0_q0     1.0     0.0     0.0     0.0     0.0     0.0
ow0_q1    -1.0     1.0     0.0     0.0     0.0     0.0
ow0_q2     0.0    -1.0     1.0     0.0     0.0     0.0
ow0_q3     0.0     0.0    -1.0     1.0     0.0     0.0
ow0_q4     0.0     0.0     0.0    -1.0     1.0     0.0
ow0_q5     0.0     0.0     0.0     0.0    -1.0     1.0 

G:
ow0_q0    1125.000
ow0_q1     630.000
ow0_q2     630.000
ow0_q3      30.375
ow0_q4      30.375
ow0_q5     360.000
dtype: float64 

C:
ow0_θ0           0.0
ow0_θ1    18216000.0
ow0_θ2           0.0
ow0_θ3      239580.0
ow0_θ4           0.0
ow0_θ5           0.0
dtype: float64 

b:
ow0_q0     To
ow0_q1    0.0
ow0_q2    0.0
ow0_q3    0.0
ow0_q4    0.0
ow0_q5    0.0
dtype: object 

f:
ow0_θ0     Φo
ow0_θ1    0.0
ow0_θ2    0.0
ow0_θ3    0.0
ow0_θ4     Φi
ow0_θ5    0.0
dtype: object 

y:
ow0_θ0    0.0
ow0_θ1    0.0
ow0_θ2    0.0
ow0_θ3    0.0
ow0_θ4    0.0
ow0_θ5    1.0
dtype: float64 


### Auto-numbering

There are two types of files describing the thermal circuits:
1. walls, described by the files `wall_types.csv` and `walls_.csv`.
2. thermal circuits, described by the files `TC0.csv`, `TC1.csv`, etc.

The walls are auto-numbered, i.e., the nodes and oriented branches are numbered automatically and named **θ0, θ1**, ... and **q0, q1**, ..., respectively, preceeded by the name of the wall, e.g., **ow0_**.

In [18]:
TCd['ow0']['A']

,ow0_θ0,ow0_θ1,ow0_θ2,ow0_θ3,ow0_θ4,ow0_θ5
ow0_q0,1.0,0.0,0.0,0.0,0.0,0.0
ow0_q1,-1.0,1.0,0.0,0.0,0.0,0.0
ow0_q2,0.0,-1.0,1.0,0.0,0.0,0.0
ow0_q3,0.0,0.0,-1.0,1.0,0.0,0.0
ow0_q4,0.0,0.0,0.0,-1.0,1.0,0.0
ow0_q5,0.0,0.0,0.0,0.0,-1.0,1.0


The thermal circuits can be autonumbered or they can keep the symbols given to the nodes and the branches in the `TC... .csv` files.

In [19]:
print('\nDisassembled thermal circuit with autonumbering')
TCd = pd_dm4bem.bldg2TCd(folder_path='bldg',
                        TC_auto_number=True)
TCd['c1']['A']


Disassembled thermal circuit with autonumbering


,c1_θ0,c1_θ1,c1_θ2
c1_q0,1.0,0.0,0.0
c1_q1,-1.0,1.0,0.0
c1_q2,0.0,-1.0,1.0


In [20]:
print('\nDisassembled thermal circuit with symbols')
TCd = pd_dm4bem.bldg2TCd(folder_path='bldg',
                        TC_auto_number=False)
TCd['c1']['A']


Disassembled thermal circuit with symbols


,c1_θg,c1_θwi,c1_θai
c1_qgo,1.0,0.0,0.0
c1_qgi,-1.0,1.0,0.0
c1_qai,0.0,-1.0,1.0
